In [1]:
import warnings
import numpy as np
import string
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import keras
import os
import re
from collections import defaultdict
from py_translator import Translator


from textblob import TextBlob, Word
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag

from sklearn import random_projection
from sklearn import cross_validation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection, naive_bayes, svm

from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer, word_tokenize

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, CuDNNGRU, CuDNNLSTM, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten, Reshape
from keras.layers import Conv1D, Conv2D, Convolution2D, MaxPool2D, MaxPooling2D, Flatten, Reshape, BatchNormalization, Concatenate
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model

from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras.optimizers import Adam
from keras.regularizers import l1, l2

# from tensorflow.contrib.cudnn_rnn.python.ops import cudnn_rnn_ops

from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.layers.pooling import GlobalAveragePooling1D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import Model
from keras import metrics
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

stop_words = set(stopwords.words('english') + list(string.punctuation))

warnings.filterwarnings('ignore')

/Users/ivan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/ivan/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# 1. Data Reading

In [2]:
Train = pd.read_csv("./data/train.csv")
Valid = pd.read_csv("./data/valid.csv")

In [3]:
Train.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,dsvFUqrhytVmKXCW7bKNhA,0,2013-03-10 01:24:07,0,58nqw-MdO6EDACPaKDM59Q,5.0,"As a student, by back and neck are under const...",0,1NImHsg1kc76n_cQyTm0bg
1,6YrO-hJNof4wsx4f0YQ8yg,0,2014-12-22 17:06:23,0,eVeQtMGaB5tdCH0hKJwxKw,3.0,Stayed here for a football game at University ...,0,Oi0G3jFm2jtG2W02dZTdEQ
2,LtyoPfxpvcF_9e9wMoUi0w,0,2016-04-25 09:55:57,0,AHMHUbq0eAUOuOPFoeO5iw,4.0,"Very good salads, generous portions. I either ...",1,4jbz7cOVuV_Q7v2b3pNrLw
3,QzvLnOqwH6BIY_jCOvzuQQ,0,2017-04-06 04:18:48,0,8RDvwneSMsbpmi5UgGq60A,1.0,The experience I had with this company growing...,0,gJgPs0QXE587T9SIR6NCdQ
4,tPaweigPsXacvQT8daYT4g,0,2017-03-22 15:38:59,0,IboCoxoL0IFtJnsGPHCugg,5.0,"Easiest furniture purchase, with a great price...",0,zp8XLBJmgw55ZTlp5raK7Q


In [35]:
Train['text'] = Train['text'].apply(lambda row:' '.join(list(row)))

In [4]:
df = pd.concat([Train, Valid])
df = df.reset_index(drop=True)
df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,dsvFUqrhytVmKXCW7bKNhA,0,2013-03-10 01:24:07,0,58nqw-MdO6EDACPaKDM59Q,5.0,"As a student, by back and neck are under const...",0,1NImHsg1kc76n_cQyTm0bg
1,6YrO-hJNof4wsx4f0YQ8yg,0,2014-12-22 17:06:23,0,eVeQtMGaB5tdCH0hKJwxKw,3.0,Stayed here for a football game at University ...,0,Oi0G3jFm2jtG2W02dZTdEQ
2,LtyoPfxpvcF_9e9wMoUi0w,0,2016-04-25 09:55:57,0,AHMHUbq0eAUOuOPFoeO5iw,4.0,"Very good salads, generous portions. I either ...",1,4jbz7cOVuV_Q7v2b3pNrLw
3,QzvLnOqwH6BIY_jCOvzuQQ,0,2017-04-06 04:18:48,0,8RDvwneSMsbpmi5UgGq60A,1.0,The experience I had with this company growing...,0,gJgPs0QXE587T9SIR6NCdQ
4,tPaweigPsXacvQT8daYT4g,0,2017-03-22 15:38:59,0,IboCoxoL0IFtJnsGPHCugg,5.0,"Easiest furniture purchase, with a great price...",0,zp8XLBJmgw55ZTlp5raK7Q


# 2. Pre-processing

### 2.1 Rare words removal

In [7]:
freq = pd.Series(' '.join(df['text']).split()).value_counts()[-10:]
freq

boozy.         1
laterrrr       1
CONs--Small    1
Cherries.      1
biers          1
three...       1
1500.00.       1
Valley.)       1
features).     1
gruel.         1
dtype: int64

In [ ]:
freq = list(freq.index)
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

### 2.2 Lemmatization

In [ ]:
df['text'] = [word_tokenize(entry) for entry in df['text']]

In [ ]:
tag_map = defaultdict(lambda : wordnet.NOUN)
tag_map['J'] = wordnet.ADJ
tag_map['V'] = wordnet.VERB
tag_map['R'] = wordnet.ADV

In [ ]:
def lemmatization(df):
    for index,entry in enumerate(df['text']):
        # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
        # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
        # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
            # Below condition is to check for Stop words and consider only alphabets
            if word not in stopwords.words('english') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
        # The final processed set of words for each iteration will be stored in 'text_final'
        df.loc[index,'text_final'] = str(Final_words)

In [ ]:
lemmatization(df)

In [ ]:
df.to_csv("pre_processed_df.csv", index=False)

# 3. Preview Preprocessed Data

In [5]:
df = pd.read_csv("./data/pre_processed_df.csv")

In [6]:
df.head()

,text,stars,word_count,char_count,avg_word,stopwords,hastags,numerics,upper,text_final
0,"['student', 'back', 'neck', 'constant', 'strai...",5.0,48,295,5.166667,14,0,0,2,"['student', 'back', 'neck', 'constant', 'strai..."
1,"['stayed', 'football', 'game', 'university', '...",3.0,37,218,4.918919,13,0,0,0,"['stay', 'football', 'game', 'university', 'ph..."
2,"['good', 'salads', 'generous', 'portions', 'ei...",4.0,19,106,4.631579,4,0,0,2,"['good', 'salad', 'generous', 'portion', 'eith..."
3,"['experience', 'company', 'growing', 'awesome'...",1.0,134,716,4.343284,55,0,2,7,"['experience', 'company', 'grow', 'awesome', '..."
4,"['easiest', 'furniture', 'purchase', 'great', ...",5.0,19,122,5.473684,6,0,0,0,"['easy', 'furniture', 'purchase', 'great', 'pr..."


# 4. TFIDF

In [7]:
tfidf = TfidfVectorizer(max_features=10000, lowercase=True,
                        stop_words='english', ngram_range=(1, 2))

vect_train = tfidf.fit_transform(df['text_final'].iloc[:100000])
vect_test = tfidf.transform(df['text_final'].iloc[100000:])
vocab = tfidf.vocabulary_
print(len(vocab))

10000


In [8]:
vect_train.shape

(100000, 10000)

In [9]:
# Show the Model as a pandas DataFrame
feature_names = tfidf.get_feature_names()
vectdf = pd.DataFrame(vect_train.toarray(), columns = feature_names)
vectdf.head()

,aaa,aaron,ability,able,able come,able eat,able enjoy,able help,able make,able order,...,zen,zero,zero star,zip,zombie,zone,zoo,zucchini,ça,était
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 5. Adding and Normalising features

In [10]:
Train['useful_nom'] = (Train['useful']-min(Train['useful']))/(max(Train['useful'])-min(Train['useful']))
Train['funny_nom'] = (Train['funny']-min(Train['funny']))/(max(Train['funny'])-min(Train['funny']))
Train['cool_nom'] = (Train['cool']-min(Train['cool']))/(max(Train['cool'])-min(Train['cool']))

In [11]:
Valid['useful_nom'] = (Valid['useful']-min(Valid['useful']))/(max(Valid['useful'])-min(Valid['useful']))
Valid['funny_nom'] = (Valid['funny']-min(Valid['funny']))/(max(Valid['funny'])-min(Valid['funny']))
Valid['cool_nom'] = (Valid['cool']-min(Valid['cool']))/(max(Valid['cool'])-min(Valid['cool']))

In [12]:
from scipy.sparse import csr_matrix, hstack
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [13]:
useful_train = Train['useful_nom']
useful_test = Valid['useful_nom']
funny_train = Train['funny_nom']
funny_test = Valid['funny_nom']
cool_train = Train['cool_nom']
cool_test = Valid['cool_nom']

In [14]:
vect_train2 = add_feature(vect_train, useful_train)
vect_train2 = add_feature(vect_train2, funny_train)
vect_train2 = add_feature(vect_train2, cool_train)

In [15]:
vect_test2 = add_feature(vect_test, useful_test)
vect_test2 = add_feature(vect_test2, funny_test)
vect_test2 = add_feature(vect_test2, cool_test)

In [16]:
print(vect_test2.shape)
print(vect_train2.shape)

(10000, 10003)
(100000, 10003)


# 7. Models

### 7.1 Logitisic Regression

In [17]:
Train_y = df['stars'].iloc[:100000]
Test_y = df['stars'].iloc[100000:]

In [18]:
def train_predict_model(classifier, 
                        train_features, train_labels, 
                        test_features, test_labels):
    # build model    
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features) 
    return predictions

In [23]:
from sklearn.linear_model import SGDClassifier, LogisticRegression

lr = LogisticRegression(penalty='l2', max_iter=100, C=1)
lr_bow_predictions = train_predict_model(classifier=lr, 
                                             train_features=vect_train2, train_labels=Train_y,
                                             test_features=vect_test2, test_labels=Test_y)

print('Test Accuracy:', accuracy_score(lr_bow_predictions, Test_y)*100)

Test Accuracy: 65.11


### 7.2 SVM

In [19]:
Train_x = vect_train2
Test_x = vect_test2

In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=5, gamma='auto')
k_fold_scores = cross_validation.cross_val_score(SVM, Train_x, Train_y, cv=5)

SVM.fit(Train_x,Train_y)
# predict the labels on validation dataset
predictions_SVM_train = SVM.predict(Train_x)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM_train, Train_y)*100)

predictions_SVM_test = SVM.predict(Test_x)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM_test, Test_y)*100)


In [21]:
from sklearn.externals import joblib 

#joblib.dump(SVM, 'SVM.pkl')

In [22]:
SVM_loaded = joblib.load('SVM.pkl')


predictions_SVM_loaded_test = SVM_loaded.predict(Test_x)
# Use accuracy_score function to get the accuracy
print("SVM_loaded Accuracy Score -> ",accuracy_score(predictions_SVM_loaded_test, Test_y)*100)


SVM_loaded Accuracy Score ->  41.72


### 7.3 GaussianNB and MultinomialNB

In [25]:
classifierGNB = GaussianNB()
classifierGNB.fit(vect_train2.toarray(), Train_y)
GNB_y_train_pred = classifierGNB.predict(vect_train2.toarray())
GNB_y_test_pred = classifierGNB.predict(vect_test2.toarray())

print("GaussianNB Accuracy (Train):",accuracy_score(Train_y, GNB_y_train_pred))
print("GaussianNB Accuracy (Test):",accuracy_score(Test_y, GNB_y_test_pred))

# Model Generation Using Multinomial Naive Bayes
clf = MultinomialNB().fit(vect_train2, Train_y)
MGNB_y_train_pred = classifierGNB.predict(vect_train2.toarray())
MGNB_y_test_pred = clf.predict(vect_test2.toarray())

print("MultinomialNB Accuracy (Train):",accuracy_score(Train_y, MGNB_y_train_pred))
print("MultinomialNB Accuracy (Test):",accuracy_score(Test_y, MGNB_y_test_pred))

GaussianNB Accuracy (Train): 0.55613
GaussianNB Accuracy (Test): 0.4787
MultinomialNB Accuracy (Train): 0.55613
MultinomialNB Accuracy (Test): 0.6122
